# train 기간까지 수출이 없다가 이후에 새로 수출이 생긴 아이템 리스트

In [91]:
import os
import pickle
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='AppleGothic')
plt.rc('axes', unicode_minus=False)

import warnings
warnings.filterwarnings('ignore')

In [92]:
data = pd.read_csv('../processed/export_diff_label.csv')
item_name_code = pd.read_csv('../processed/item_name-code.csv')

In [93]:
data.head(3)

,year,country,품목코드,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,label
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
1,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
2,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0


In [94]:
data.rename(columns={'country':'country_id'}, inplace=True)
data.rename(columns={'품목코드':'item_id'}, inplace=True)

In [95]:
data.isnull().sum()

year              0
country_id        0
item_id           0
수출금액              0
GDP            9448
preference     9448
GDP_growth    18612
population     9448
pop_growth    18612
USD               0
JPY               0
EUR               0
CNY               0
WTI               0
label             0
dtype: int64

In [96]:
display(data.head(3))
display(item_name_code.head(3))

,year,country_id,item_id,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,label
0,2012,Afghanistan,3.0,1.0,20203572.96,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
1,2012,Afghanistan,7.0,2.0,20203572.96,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
2,2012,Afghanistan,10.0,12.0,20203572.96,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0


,품목명,품목코드
0,살아 있는 동물,1.0
1,육과 식용 설육(屑肉),2.0
2,어류ㆍ갑각류ㆍ연체동물과 그 밖의 수생(水生) 무척추동물,3.0


In [97]:
item_name_code.shape

(96, 2)

In [98]:
data_12_18 = data.query('year < 2019')
data_19_21 = data.query('year >= 2019')
print(data_12_18.year.unique())
print(data_19_21.year.unique())

[2012 2013 2014 2015 2016 2017 2018]
[2019 2020 2021 2022]


In [99]:
data_12_18

,year,country_id,item_id,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,label
0,2012,Afghanistan,3.0,1.0,2.020357e+07,4.949620e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
1,2012,Afghanistan,7.0,2.0,2.020357e+07,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
2,2012,Afghanistan,10.0,12.0,2.020357e+07,5.939543e-07,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
3,2012,Afghanistan,11.0,2.0,2.020357e+07,9.899239e-08,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
4,2012,Afghanistan,16.0,89.0,2.020357e+07,4.405161e-06,NaN,30466479.0,NaN,1126.43,1412.96,1447.58,178.52,94.051667,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65653,2018,Zimbabwe,87.0,2594.0,3.415607e+07,7.594551e-05,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0
65654,2018,Zimbabwe,90.0,287.0,3.415607e+07,8.402606e-06,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0
65655,2018,Zimbabwe,94.0,10.0,3.415607e+07,2.927737e-07,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0
65656,2018,Zimbabwe,95.0,13.0,3.415607e+07,3.806058e-07,0.942353,15052184.0,0.020411,1101.08,997.07,1299.65,166.48,64.544167,0


In [100]:
display(sorted(data_12_18.query('country_id == "Afghanistan" & year == 2014').item_id.unique()))
display(data_12_18.query('country_id == "Afghanistan" & year==2014').item_id.nunique())

[3.0,
 7.0,
 8.0,
 10.0,
 16.0,
 19.0,
 20.0,
 21.0,
 22.0,
 24.0,
 27.0,
 29.0,
 30.0,
 33.0,
 38.0,
 39.0,
 40.0,
 48.0,
 49.0,
 52.0,
 53.0,
 54.0,
 55.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 69.0,
 70.0,
 72.0,
 73.0,
 82.0,
 84.0,
 85.0,
 87.0,
 88.0,
 90.0,
 94.0,
 95.0,
 96.0]

43

### train 기간 내 해당 국가에서 수출이 없는 아이템 데이터셋 생성  

In [101]:
# 전체 아이템 품목 집합
item_set = set(item_name_code.품목코드.tolist())
len(item_set)
print(item_set)

{1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 94.0, 95.0, 96.0, 97.0, 99.0}


In [102]:
def get_nonExp(df):
    nonExp_list = list(item_set.difference(set(df.item_id.unique()))) # 해당 연도, 국가에 없는 품목코드
    return nonExp_list

nonExp = data_12_18.groupby(['country_id','year']).progress_apply(get_nonExp).to_frame().reset_index()
print(type(nonExp))
display(nonExp)

100%|██████████| 1411/1411 [00:00<00:00, 28505.74it/s]

<class 'pandas.core.frame.DataFrame'>


,country_id,year,0
0,Afghanistan,2012,"[1.0, 2.0, 4.0, 5.0, 6.0, 8.0, 9.0, 12.0, 13.0..."
1,Afghanistan,2013,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ..."
2,Afghanistan,2014,"[1.0, 2.0, 4.0, 5.0, 6.0, 9.0, 11.0, 12.0, 13...."
3,Afghanistan,2015,"[1.0, 2.0, 4.0, 5.0, 6.0, 8.0, 9.0, 10.0, 11.0..."
4,Afghanistan,2016,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ..."
...,...,...,...
1406,Zimbabwe,2014,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ..."
1407,Zimbabwe,2015,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ..."
1408,Zimbabwe,2016,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ..."
1409,Zimbabwe,2017,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ..."


In [103]:
nonExp.rename(columns={0:'nonExp_items'}, inplace=True)
nonExp = nonExp.explode('nonExp_items')
nonExp

,country_id,year,nonExp_items
0,Afghanistan,2012,1.0
0,Afghanistan,2012,2.0
0,Afghanistan,2012,4.0
0,Afghanistan,2012,5.0
0,Afghanistan,2012,6.0
...,...,...,...
1410,Zimbabwe,2018,89.0
1410,Zimbabwe,2018,91.0
1410,Zimbabwe,2018,92.0
1410,Zimbabwe,2018,97.0


In [104]:
ref = data_12_18.groupby(['country_id','year'],as_index=False)[['GDP_growth','pop_growth']].mean()
ref

,country_id,year,GDP_growth,pop_growth
0,Afghanistan,2012,NaN,NaN
1,Afghanistan,2013,0.017864,0.035276
2,Afghanistan,2014,-0.000676,0.037253
3,Afghanistan,2015,-0.026881,0.031706
4,Afghanistan,2016,-0.098939,0.026152
...,...,...,...,...
1406,Zimbabwe,2014,0.021188,0.022156
1407,Zimbabwe,2015,0.023985,0.021593
1408,Zimbabwe,2016,0.029332,0.021036
1409,Zimbabwe,2017,-0.144232,0.020646


In [105]:
nonExp_df = nonExp.merge(ref, on=['country_id','year'], how='left')
nonExp_df

,country_id,year,nonExp_items,GDP_growth,pop_growth
0,Afghanistan,2012,1.0,NaN,NaN
1,Afghanistan,2012,2.0,NaN,NaN
2,Afghanistan,2012,4.0,NaN,NaN
3,Afghanistan,2012,5.0,NaN,NaN
4,Afghanistan,2012,6.0,NaN,NaN
...,...,...,...,...,...
69793,Zimbabwe,2018,89.0,0.942353,0.020411
69794,Zimbabwe,2018,91.0,0.942353,0.020411
69795,Zimbabwe,2018,92.0,0.942353,0.020411
69796,Zimbabwe,2018,97.0,0.942353,0.020411


In [106]:
nonExp.shape, nonExp_df.shape, nonExp_df.drop_duplicates().shape

((69798, 3), (69798, 5), (69798, 5))

In [107]:
nonExp_df['preference'] = 0
nonExp_df

,country_id,year,nonExp_items,GDP_growth,pop_growth,preference
0,Afghanistan,2012,1.0,NaN,NaN,0
1,Afghanistan,2012,2.0,NaN,NaN,0
2,Afghanistan,2012,4.0,NaN,NaN,0
3,Afghanistan,2012,5.0,NaN,NaN,0
4,Afghanistan,2012,6.0,NaN,NaN,0
...,...,...,...,...,...,...
69793,Zimbabwe,2018,89.0,0.942353,0.020411,0
69794,Zimbabwe,2018,91.0,0.942353,0.020411,0
69795,Zimbabwe,2018,92.0,0.942353,0.020411,0
69796,Zimbabwe,2018,97.0,0.942353,0.020411,0


In [108]:
nonExp_df.shape, nonExp_df.drop_duplicates().shape

((69798, 6), (69798, 6))

In [109]:
print(sorted(data_12_18.query('country_id == "Afghanistan" & year == 2013').item_id.unique()))
print(sorted(nonExp_df.query('country_id == "Afghanistan" & year == 2013').nonExp_items.unique()))
print()
print(data_12_18.query('country_id == "Afghanistan" & year==2013').item_id.nunique())
print(nonExp_df.query('country_id == "Afghanistan" & year==2013').nonExp_items.nunique())

[10.0, 16.0, 19.0, 20.0, 21.0, 22.0, 24.0, 27.0, 28.0, 29.0, 30.0, 38.0, 39.0, 40.0, 49.0, 52.0, 54.0, 55.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 68.0, 73.0, 76.0, 82.0, 83.0, 84.0, 85.0, 87.0, 88.0, 90.0, 94.0, 95.0, 96.0]
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 11.0, 12.0, 13.0, 14.0, 15.0, 17.0, 18.0, 23.0, 25.0, 26.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 50.0, 51.0, 53.0, 56.0, 57.0, 66.0, 67.0, 69.0, 70.0, 71.0, 72.0, 74.0, 75.0, 78.0, 79.0, 80.0, 81.0, 86.0, 89.0, 91.0, 92.0, 97.0, 99.0]

39
57


In [110]:
nonExp_df.isnull().sum()

country_id          0
year                0
nonExp_items        0
GDP_growth      10324
pop_growth      10324
preference          0
dtype: int64

In [111]:
nonExp_df[nonExp_df.nonExp_items.isnull()]

,country_id,year,nonExp_items,GDP_growth,pop_growth,preference


In [112]:
# 데이터프레임 저장
nonExp_df.to_csv('../processed/nonExp_12_18.csv', index=False)

### train 기간까지 수출이 없다가 이후에 새로 수출이 생긴 아이템 리스트

In [113]:
display(nonExp_df.head(3))
display(data_19_21.head(3))

,country_id,year,nonExp_items,GDP_growth,pop_growth,preference
0,Afghanistan,2012,1.0,NaN,NaN,0
1,Afghanistan,2012,2.0,NaN,NaN,0
2,Afghanistan,2012,4.0,NaN,NaN,0


,year,country_id,item_id,수출금액,GDP,preference,GDP_growth,population,pop_growth,USD,JPY,EUR,CNY,WTI,label
65658,2019,Afghanistan,12.0,24.0,1.890449e+07,0.000001,0.026367,37769499.0,0.029512,1166.51,1070.63,1305.86,168.68,57.099167,0
65659,2019,Afghanistan,19.0,240.0,1.890449e+07,0.000013,0.026367,37769499.0,0.029512,1166.51,1070.63,1305.86,168.68,57.099167,1
65660,2019,Afghanistan,21.0,518.0,1.890449e+07,0.000027,0.026367,37769499.0,0.029512,1166.51,1070.63,1305.86,168.68,57.099167,0


In [116]:
# Afghanistan, 2013년에 수출 없는 아이템
no_12 = set(nonExp_df[(nonExp_df['country_id'] == "Afghanistan") & (nonExp_df['year'] == 2012)].nonExp_items.unique())

# Afghanistan, 2013~2015년에 수출한 아이템
yes_13_15 = set(data[(data['country_id'] == "Afghanistan") & (data['year'].isin(range(2013,2016)))].item_id.unique())

# Afghanistan, 2013년에 수출 없지만 2014~2016년에 수출한 아이템
new_13_15 = no_12.intersection(yes_13_15)
print(len(new_13_15))
print(new_13_15)

17
{64.0, 33.0, 68.0, 69.0, 70.0, 8.0, 72.0, 42.0, 76.0, 48.0, 53.0, 57.0, 91.0, 28.0, 29.0, 94.0, 95.0}


In [117]:
print(no_12)
print(yes_13_15)

{1.0, 2.0, 4.0, 5.0, 6.0, 8.0, 9.0, 12.0, 13.0, 14.0, 15.0, 18.0, 23.0, 25.0, 26.0, 28.0, 29.0, 31.0, 33.0, 34.0, 35.0, 36.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 50.0, 51.0, 53.0, 57.0, 64.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 74.0, 75.0, 76.0, 78.0, 79.0, 80.0, 81.0, 89.0, 91.0, 92.0, 94.0, 95.0, 97.0, 99.0}
{3.0, 7.0, 8.0, 10.0, 16.0, 19.0, 20.0, 21.0, 22.0, 24.0, 27.0, 28.0, 29.0, 30.0, 33.0, 38.0, 39.0, 40.0, 42.0, 48.0, 49.0, 52.0, 53.0, 54.0, 55.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 68.0, 69.0, 70.0, 72.0, 73.0, 76.0, 82.0, 83.0, 84.0, 85.0, 87.0, 88.0, 90.0, 91.0, 94.0, 95.0, 96.0}


In [118]:
# 금년 수출 없지만 3년 이내에 수출한 아이템 
new_exp_3years = {}
for country in tqdm(nonExp_df.country_id.unique()):
    for year in range(2012, 2019):  # 마지막 2018년: 2019, 2020, 2021년 이내에 수출한 아이템
        no_exp = set(nonExp_df[(nonExp_df['country_id']==country)&(nonExp_df['year'] == year)].nonExp_items.unique())
        exp_3years = set(data[(data['country_id']==country)&(data['year'].isin(range(year+1,year+4)))].item_id.unique())
        new_exp = list(no_exp.intersection(exp_3years))
        new_exp_3years[(country, year)] = new_exp
print(new_exp_3years)

100%|██████████| 203/203 [00:06<00:00, 30.17it/s]

{('Afghanistan', 2012): [64.0, 33.0, 68.0, 69.0, 70.0, 8.0, 72.0, 42.0, 76.0, 48.0, 53.0, 57.0, 91.0, 28.0, 29.0, 94.0, 95.0], ('Afghanistan', 2013): [33.0, 3.0, 69.0, 70.0, 7.0, 8.0, 71.0, 42.0, 72.0, 74.0, 47.0, 48.0, 53.0, 56.0, 57.0, 91.0], ('Afghanistan', 2014): [32.0, 71.0, 42.0, 74.0, 47.0, 83.0, 56.0, 57.0, 91.0, 28.0], ('Afghanistan', 2015): [32.0, 33.0, 34.0, 71.0, 74.0, 12.0, 13.0, 47.0, 49.0, 18.0, 83.0, 56.0, 58.0], ('Afghanistan', 2016): [32.0, 96.0, 34.0, 37.0, 12.0, 13.0, 44.0, 48.0, 18.0, 19.0, 20.0, 83.0, 24.0, 28.0], ('Afghanistan', 2017): [96.0, 34.0, 37.0, 74.0, 12.0, 13.0, 44.0, 18.0, 19.0, 20.0, 56.0, 28.0], ('Afghanistan', 2018): [37.0, 44.0, 49.0, 19.0, 28.0], ('Albania', 2012): [32.0, 34.0, 3.0, 70.0, 71.0, 42.0, 44.0, 76.0, 82.0, 55.0, 24.0, 61.0, 62.0, 95.0], ('Albania', 2013): [32.0, 35.0, 67.0, 68.0, 71.0, 42.0, 44.0, 76.0, 48.0, 82.0, 55.0, 24.0, 61.0, 62.0], ('Albania', 2014): [32.0, 64.0, 34.0, 70.0, 71.0, 44.0, 76.0, 79.0, 49.0, 55.0, 24.0, 61.0, 62.0,

In [119]:
print(new_exp_3years)

{('Afghanistan', 2012): [64.0, 33.0, 68.0, 69.0, 70.0, 8.0, 72.0, 42.0, 76.0, 48.0, 53.0, 57.0, 91.0, 28.0, 29.0, 94.0, 95.0], ('Afghanistan', 2013): [33.0, 3.0, 69.0, 70.0, 7.0, 8.0, 71.0, 42.0, 72.0, 74.0, 47.0, 48.0, 53.0, 56.0, 57.0, 91.0], ('Afghanistan', 2014): [32.0, 71.0, 42.0, 74.0, 47.0, 83.0, 56.0, 57.0, 91.0, 28.0], ('Afghanistan', 2015): [32.0, 33.0, 34.0, 71.0, 74.0, 12.0, 13.0, 47.0, 49.0, 18.0, 83.0, 56.0, 58.0], ('Afghanistan', 2016): [32.0, 96.0, 34.0, 37.0, 12.0, 13.0, 44.0, 48.0, 18.0, 19.0, 20.0, 83.0, 24.0, 28.0], ('Afghanistan', 2017): [96.0, 34.0, 37.0, 74.0, 12.0, 13.0, 44.0, 18.0, 19.0, 20.0, 56.0, 28.0], ('Afghanistan', 2018): [37.0, 44.0, 49.0, 19.0, 28.0], ('Albania', 2012): [32.0, 34.0, 3.0, 70.0, 71.0, 42.0, 44.0, 76.0, 82.0, 55.0, 24.0, 61.0, 62.0, 95.0], ('Albania', 2013): [32.0, 35.0, 67.0, 68.0, 71.0, 42.0, 44.0, 76.0, 48.0, 82.0, 55.0, 24.0, 61.0, 62.0], ('Albania', 2014): [32.0, 64.0, 34.0, 70.0, 71.0, 44.0, 76.0, 79.0, 49.0, 55.0, 24.0, 61.0, 62.0,

In [120]:
# 저장
with open('../processed/new_exp_3years.pkl', 'wb') as f:
    pickle.dump(new_exp_3years, f)